In [ ]:
import pandas as pd
import time
from multiprocessing import Pool
from sklearn.model_selection import ParameterGrid
import csv
from utils.PreProcess import preProcessData
from utils.TimeUtil import convertTime, revConvertTime
from utils.DataUtil import parseMinuteData
from scripts.dayScript import dayScript
from classes import ParamEntry

In [23]:
def movingAverageExperiment(paramList):
    df, dateList, paramEntry = paramList
    money = 1
    for date in dateList:
        new_df = df[(df.date == date)]
        day = dayScript(new_df, paramEntry, money, verbose=False)
#         print(date, money, updatedMoney, dailyTrades)
        money = day.money

    yearlyProfitPercentage = (money - 1) * 100
    paramEntry.profitPercentage = yearlyProfitPercentage
    print(paramEntry)
   
    return paramEntry

In [24]:
def bruteAnalysis(stockName, parameterDict, pool, threadPoolSize):
    csvList = []
    parameterGrid = getParameterGrid(parameterDict)
    folderName = "IntradayData_2018"
    df, dateList = preProcessData(folderName, stockName, parameterDict.shortTermList, parameterDict.longTermList, 'exponential')
    parameterGridSize = len(parameterGrid)
    print('param combination = ' , parameterGridSize)
    i=0
    while i<parameterGridSize:
        paramEntryList =[]
        for j in range(threadPoolSize):
            if i+j < parameterGridSize:
                params = parameterGrid[i+j]
                paramEntry = ParamEntry(params, stockName)
                paramEntryList.append([df, dateList, paramEntry])
        start = time.time()
        resultList = pool.map(movingAverageExperiment, paramEntryList)
        end = time.time()
        for paramEntry in resultList:
            csvList.append(paramEntry.getCsvPoint())
        print('Time - ',end - start)
        i+=threadPoolSize
        print(i,len(grid))
        break
    return csvList


In [25]:
# stockList = ['ACC_F1','ASHOKLEY_F1','AXISBANK_F1','BHARTIARTL_F1','RELIANCE_F1','INFY_F1','WIPRO_F1','PNB_F1','SBIN_F1','SUNPHARMA_F1','GRASIM_F1','LUPIN_F1','LT_F1','HINDUNILVR_F1']
stockList = ['LUPIN_F1', 'ASHOKLEY_F1','ACC_F1']

In [26]:
def getParameterGrid(parameterDict):
    parameterGrid = ParameterGrid(parameterDict)
    return parameterGrid

def getParameterDict():
    # # #old
    shortTerm = [1,4,12,20]
    longTerm = [100,200,500,1000]
    targetPercentage = [0.01, 0.05, 0.075]
    stopLossPercentage = [0.002, 0.006, 0.008, 0.010]
    entryDifference = [0.00001, 0.00005, 0.00010, 0.0002]
    # # #new
    # shortTerm = [4,8,12,16]
    # longTerm = [100,200,300,400,500]
    # targetPercentage = [0.01, 0.05, 0.075]
    # stopLossPercentage = [0.006, 0.007, 0.008, 0.010]
    # entryDifference = [0.00005, 0.000075, 0.0001]
    # # # #v3
    # shortTerm = [8]
    # longTerm = [200]
    # targetPercentage = [0.01, 0.05, 0.075]
    # stopLossPercentage = [0.006, 0.007, 0.008, 0.010]
    # entryDifference = [0.1]
    parameterDict = {
                    'shortTerm' : shortTerm, 
                    'longTerm': longTerm, 
                    'targetPercentage': targetPercentage, 
                    'stopLossPercentage': stopLossPercentage, 
                    'entryDifference':entryDifference
                }
    return parameterDict


In [28]:
threadPoolSize = 24
parameterDict = getParameterDict()
pool = Pool(threadPoolSize)
with open('result.csv', 'w') as f_out:
    out_colnames = ["stockName","shortTerm", "longTerm", "targetPercentage", "stopLossPercentage", "entryDifference","profitPercentage"]
    csv_writer = csv.DictWriter(f_out, fieldnames = out_colnames)
    csv_writer.writeheader()
    for stock in stockList:
        csvList = bruteAnalysis(stock[:-3], parameterDict, pool, threadPoolSize)
        for point in csvList:
            csv_writer.writerow(point)
        break
pool.terminate()
pool.join()

('param combination = ', 768)


UnboundLocalError: local variable 'timeEntered' referenced before assignment